In [2]:
import win32com.client as client
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
import time
import datetime as dt
from datetime import datetime
from PIL import Image
import os
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import win32gui
import win32con
import pandas as pd
import warnings
import smartsheet
import traceback

# Função para clicar em elementos padrão
def click(campo):
    time.sleep(3)
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
    element.click()

# Função para dar 2 cliques em elementos padrão
def dois_click(campo):
    time.sleep(3)
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
    actions = ActionChains(driver)
    actions.double_click(element)
    actions.perform()

# Função para escrever em campos input
def escrever(campo, texto):
    time.sleep(3)
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, campo)))
        element.clear()
        element.send_keys(texto)
    except TimeoutException:
        print("Timeout: Elemento de entrada não encontrado. Ignorando e continuando.")

# Função que considera tempo de execução
def countdown(t): 
    while t: 
        mins, secs = divmod(t, 60) 
        timer = '{:02d}:{:02d}'.format(mins, secs) 
        print(timer, end="\r") 
        time.sleep(1) 
        t -= 1


# Crie um data frame a partir dos dados da planilha do excel
def carrega_dados(sheet):
    col_names = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    data_frame = pd.DataFrame(rows, columns=col_names)
    return data_frame

def update_smartsheet(column_title, cell_value, row_index):

    #Definindo veiraveis globais para reutilizar posteriormente
    global smart
    global sheet_id
    global max_rows
    
    # Testa se a index da linha é menor que o quantidade de linhas
    if row_index < max_rows:
        try:
            #Assume valores da planilha a partir do id da planilha do Smartsheet 
            sheet = smart.Sheets.get_sheet(sheet_id)
            rows = sheet.rows
            row_id = rows[row_index].id
            column_id = [col.id for col in sheet.columns if col.title == column_title][0]
            cell = smartsheet.models.Cell()
            cell.column_id = column_id
            
            #atualiza valor da célula conforme o titulo da coluna e o row_index
            cell.value = cell_value
            updated_row = smartsheet.models.Row()
            updated_row.id = row_id
            updated_row.cells.append(cell)
            response = smart.Sheets.update_rows(sheet_id, [updated_row])
            
            #Printa Valores dependendo do sucesso na requisição da API
            if response.message == 'SUCCESS':
                print(f'Valor "{cell_value}" atualizado na coluna "{column_title}" com sucesso.')
            else:
                print('Falha ao atualizar o valor na coluna.')
        except Exception as e:
            print(f'Erro: {e}')
    else:
        print('Índice da linha fora do intervalo.')
    return row_index

def acessar_relatorio(element):
    #tela para acessar o campo de atendimento do portal
    iframe_busc = driver.find_element(By.XPATH,element)                                       
    iframe_busc.click()
    from_trat =  driver.switch_to.frame(iframe_busc)



def click_webelement(xpath, indice):
    #defzinha para clicar nos elementos usando como base o texto que eles possuem
    lista_campos = driver.find_elements(By.XPATH,xpath)#gera uma lista com todos os elementos encontrados que possuem aquele parametro
    campo_a_ser_clicado = lista_campos[indice]#armazena o elemento da lista que queremos clicar :D

    try:
        # Aguardar um pouco para que você possa ver o resultado
        time.sleep(5)
        # Aguardar até que o elemento esteja presente
        WebDriverWait(driver, 50).until(EC.visibility_of(campo_a_ser_clicado))
        campo_a_ser_clicado.click()
    except TimeoutException:
        # Se o elemento não for encontrado dentro do tempo de espera, imprimir uma mensagem
        print("Timeout: Elemento não encontrado. Ignorando e continuando.")
def login():
    driver.get("https://portal.gpssa.com.br/gps/portal.aspx")
    escrever("/html/body/form/div[1]/div/div[1]/div/div/div/div/div[4]/div/div[1]/div/div/div[2]/div[1]/div/div/input","fernando.schmidt")
    escrever("/html/body/form/div[1]/div/div[1]/div/div/div/div/div[4]/div/div[1]/div/div/div[3]/div[1]/div/div/input", "GPS@042024")
    click("/html/body/form/div[1]/div/div[1]/div/div/div/div/div[4]/div/div[2]/div/div/a/span/span/span[2]")
    dois_click("/html/body/div[1]/div[2]/div[4]/div/div/a[6]/span/span/span[2]")
    dois_click("/html/body/form/div[2]/div[3]/div/div/div/div/div/div[2]/div/div[1]/table[15]/tbody/tr/td/div/span")
    time.sleep(5)
    #clica em Gestão de pessoas
    dois_click("//*[text()='Gestão de Pessoas']")
    #clica em documentos e contratos
    dois_click("//*[text()='4. Documentos e Contratos']")
    #clica no Card de colaborador
    dois_click("//*[text()='4.1 - Card - Gestão do Colaborador']")
    #acessando o relatório do portal 
    time.sleep(10)
    acessar_relatorio("/html/body/form/div[4]/div/div/div/div/div[3]/div/div[2]/div[2]/div[1]/div/iframe")
    print("ok iframe acessado")


#Token de acesso da API Smartsheet
access_token = 'qGPQ1qbfWCpw07VytHUObAjZj5h2cAvHOBUet'
#conecta à API
smart = smartsheet.Smartsheet(access_token)
#Id da planilha Smartsheet
sheet_id = '357594556223364'


#Assume valores da Planilha 
sheet = smart.Sheets.get_sheet(sheet_id)
#Carrega para o dataframe df
df = carrega_dados(sheet)

#definindo li de inicio e fim de execução 
row_index = 3048
max_rows = len(df)

#Configurações do navegador webdriver
options = webdriver.EdgeOptions()
options.add_argument("--headless")
driver = webdriver.Edge()
driver.maximize_window()

# Define o tempo de início
start_time = time.time()

# Define a duração do loop em segundos (12 horas
duration = 480 * 60
login()
# Loop que roda por 19 minutos
while time.time() - start_time < duration:
    print("--------------INICIOU O PROCESSO!--------------")
    try:

    #     # Recarregar o DataFrame a cada iteração
        sheet = smart.Sheets.get_sheet(sheet_id)
        df = carrega_dados(sheet)
        max_rows = len(df)            
        
        # Testa se a index da linha é menor que o quantidade de linhas
        if row_index >= max_rows:
            row_index = 0
            
        #Definindo variáveis conforme o row_index
        #Assume nome
        nome = int(df['CPF'].iloc[row_index])
        print(nome)
        link_whats = df['link_whats'].iloc[row_index]
        CR = df['CR'].iloc[row_index]

        
        if CR != "Enviado":
            
            print("--------------INICIANDO AUTOMAÇÃO--------------")
            
            data_atual =  dt.datetime.today() 
            data_limite = data_atual + dt.timedelta(days=10)
            data_limte_format = data_limite.strftime('%d/%m/%Y')
            
            #foca para a primeira aba do navegador
            browser_tabs = driver.window_handles
            #acessa o iframe do Card}
            driver.switch_to.window(browser_tabs[0])
            acessar_relatorio("/html/body/form/div[4]/div/div/div/div/div[3]/div/div[2]/div[2]/div[1]/div/iframe")
            #pesquisando pelo nome da pessoa no Card
            escrever("//*[@ID='txtPesquisa-inputEl']", nome)
            #Pesquisando
            click("//*[@ID='btnPesquisar-btnWrap']")
            time.sleep(8)
            #clicando no campo de Lupa
            click("/html/body/div[1]/div[2]/div[3]/div/div[2]/table/tbody/tr/td[1]/div/div")
            time.sleep(8)
            #mudando o foco para a aba que acabamos de abrir
            browser_tabs = driver.window_handles
            driver.switch_to.window(browser_tabs[1])
            xpath_elemento = "/html/body/div[1]/div/div/div/div/div/div/div/div/div/div/div/div/div/div/header/div/div[4]/div/div[2]/a[2]"
            elemento_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_elemento)))
            # Obtém o valor do atributo href (o link)
            inicio_link = elemento_link.get_attribute('href')


            link = f'{inicio_link}?text=Ol%C3%A1!%0A%0AEsperamos%20que%20esteja%20bem.%0A%0AEstamos%20verificando%20se%20voc%C3%AA%20conseguiu%20sacar%20as%20duas%20parcelas%20de%20R%24%201.412%2C00%20cada%2C%20conforme%20previsto%20na%20MTE%20991%20(Apoio%20Financeiro%20-%20Ajuda%20RS%2C%20concedido%20%C3%A0s%20empresas).%0A%0APor%20favor%2C%20*apenas%20responda%20a%20enquete*.%20%0A%0ACaso%20a%20resposta%20for%20N%C3%83O%20entraremos%20em%20contato%20com%20voc%C3%AA.%0A%20'
            
            print(inicio_link)

            # Imprime o link
            driver.get(link)
            time.sleep(1)
            click('/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[2]/div[1]/a/span')
            click('/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[3]/div/div/h4[2]/a/span')
            
            time.sleep(15)
            click('//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[2]/button/span')
            
            click('//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/div/div/span')
            click('//*[@id="main"]/footer/div[1]/div/span[2]/div/div[1]/div[2]/div/span/div/ul/div/div[5]/li/div')
            escrever('/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div/div/div[1]/div/div[1]/div[2]/div[3]/div/div/p', "⬇️⬇️Responda aqui.⬇️⬇️")
            escrever('/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div/div/div[1]/div/div[2]/div/div[1]/div/div/div/div/div[2]/div[3]/div/div/p', "SIM")
            escrever('/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div/div/div[1]/div/div[2]/div/div[2]/div/div/div/div/div[2]/div[3]/div/div/p', "NÃO")
            
            click('/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div/div/div[1]/div/div[3]/div/div/div[1]')
            click('/html/body/div[1]/div/div/span[2]/div/div/div/div/div/div/div/div/div[2]/div')

            time.sleep(2)
            # Atualiza a coluna "CR" para "Enviado" na linha desejada
            update_smartsheet("CR", "Enviado", row_index) 
            
            # Atualiza a coluna "CR" para "Enviado" na linha desejada
            df.loc[row_index, ] = inicio_link
            update_smartsheet("link_whats", inicio_link, row_index) 
            # Salva o DataFrame atualizado no mesmo arquivo

            driver.close()  
            
       
        else:
            if CR == "":
                # Atualiza a coluna "CR" para "Enviado" na linha desejada
                update_smartsheet("CR", "Não Enviado", row_index) 

        row_index+=1
        
    except Exception as e:
        print(e)
        # Atualiza a coluna "CR" para "Enviado" na linha desejada
        df.loc[row_index, "CR"] = "Não Enviado"
        # Salva o DataFrame atualizado no mesmo arquivo
        driver.close()
        row_index+=1
        

ok iframe acessado
--------------INICIOU O PROCESSO!--------------
83663371034
--------------INICIANDO AUTOMAÇÃO--------------
https://wa.me/5555999908283
Message: element not interactable
  (Session info: MicrosoftEdge=127.0.2651.105)
Stacktrace:
	GetHandleVerifier [0x00007FF6F23C05A2+11762]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6F23608E9+271401]
	(No symbol) [0x00007FF6F217FCD5]
	(No symbol) [0x00007FF6F21C7C23]
	(No symbol) [0x00007FF6F21BCA4C]
	(No symbol) [0x00007FF6F21E5F3A]
	(No symbol) [0x00007FF6F21BC2FD]
	(No symbol) [0x00007FF6F21BC1CD]
	(No symbol) [0x00007FF6F21E6220]
	(No symbol) [0x00007FF6F21BC2FD]
	(No symbol) [0x00007FF6F21FFA97]
	(No symbol) [0x00007FF6F21E5BA3]
	(No symbol) [0x00007FF6F21BB80C]
	(No symbol) [0x00007FF6F21BACED]
	(No symbol) [0x00007FF6F21BB3D1]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F253B524+879700]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6F22F5D4F+56159]

{"response": {"statusCode": 500, "reason": "Internal Server Error", "content": {"errorCode": 4002, "message": "Server timeout exceeded. Request has failed.", "refId": "1p738y"}}}


'Error' object has no attribute 'columns'


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=127.0.2651.105)
Stacktrace:
	GetHandleVerifier [0x00007FF6F23C05A2+11762]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6F23608E9+271401]
	(No symbol) [0x00007FF6F217FE6E]
	(No symbol) [0x00007FF6F215B7F9]
	(No symbol) [0x00007FF6F21EC731]
	(No symbol) [0x00007FF6F21F25FD]
	(No symbol) [0x00007FF6F21E5F00]
	(No symbol) [0x00007FF6F21BB80C]
	(No symbol) [0x00007FF6F21BACED]
	(No symbol) [0x00007FF6F21BB3D1]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F253B524+879700]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6F22F5D4F+56159]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6F22E86E3+1267]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F253A27D+874925]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x000